In [1]:
from flask import Flask, render_template, request, jsonify , make_response , send_file
import re 
import ast
from flask_cors import CORS
import pandas as pd

app = Flask(__name__)
CORS(app)


In [2]:
from groq import Groq

client = Groq(
    api_key="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",
)
df = []
tst = ""
info = ""

In [3]:
def search_dataset(message) :
    # Search DATASET ...

    chat_completion = client.chat.completions.create(
    messages=[

        {
            "role": "system",
            "content": "You are a Professional Data Scientist , Search for top 5 links from kaggle to download the dataset requested by the user , get some somewhere else if it is not in Kaggle , be precise" ,
        },
        {
            "role": "user",
            "content": f"{message}" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    print(response_text)
    return response_text


In [4]:
def preprocess(mess) :

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"You are a Professional Data Scientist , Write a python code to complete the requested task , do all in inplace so that it updates in realtime" ,
            },
            {
                "role": "user", #and store the responses in a variable named as 'result'
                "content": f"This is the dataset i have : {tst} and it is loaded in a variable named as df, give a code to {mess} also call it  , give code only dont give explanation " ,
            }
        ],
        model="llama3-70b-8192",
        )
    
    # akn = client.chat.completions.create(
    #     messages=[
    #         {
    #     "role": "user",
    #     "content": "Write a success aknowledgement message in 3 lines for {mess}" ,
    # }
    #     ],
    #     model="llama3-70b-8192",
    #     )
    viss = chat_completion.choices[0].message.content
    viss =  re.sub('python', '', viss)
    viss =  re.sub('Python', '', viss)
    
    print(viss)
    pattern = r"```(.*?)```"
    local_context = {}  
    # Find all matches
    matches = re.findall(pattern, viss, re.DOTALL)
    pattern12 = r"$$(.*?)$$"
    matches12 = re.findall(pattern12, viss, re.DOTALL)

    # Safely parse the response text as a Python AST (Abstract Syntax Tree)
    parsed_ast = ast.parse(matches[0])
    result = ""
    # else:
    # Execute the parsed AST
    exec(compile(parsed_ast, filename="<ast>", mode="exec"))
    # result_value = local_context.get('result')

    return "matches12[0]",result


In [5]:
@app.route('/process_file', methods=['POST'])
def process_file():
    global info 
    global df
    global tst 
    file = request.files['file']
    if file:
        file.save('data.csv')
        df = pd.read_csv('data.csv')
        tst = df.head()
        info = df.info()
        chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"This is the dataset i have : {tst} , explain and overview of it in 2 lines" ,
            }
        ],
        model="llama3-70b-8192",
        )
        response_text = chat_completion.choices[0].message.content
        print(response_text)
        return jsonify({'response': response_text})
    else:
        return jsonify({'response': 'No file provided'})




In [6]:
# # # Process CSV 
# # @app.route('/process_data', methods=['POST'])

# def process_csv () :
#     global df
#     path = ""
#     df = pd.read_csv(path)
#     chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": f"This is the dataset i have : {tst} , explain and summarize it in 2 lines" ,
#         }
#     ],
#     model="llama3-8b-8192",
#     )
#     response_text = chat_completion.choices[0].message.content
#     print(response_text)


In [7]:
def suggest() :
    # Suggessions

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"I'm working on preprocessing a DataSet ,  This is the dataset i have : {tst} and its info : {info},give me 3 suggests to what to do further in 3 lines" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    return response_text

In [8]:
def others(mess) :

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"You are a AI data analyst , answer the below question according to your role , QUESTION : {mess}" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    return response_text

In [9]:
cnt=1

In [10]:
def visualize(mess) :
    # Visualize 
    global cnt
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            # "content": f"This is the dataset i have : {tst} , give a code to visualize column1 and column2 also call it ,mention the column names exactly , give code only dont give explanation " ,
            "content": f"This is the dataset i have : {tst} , give a code to visualize {mess}using matplotlib , also call it ,give the column names exactly as in the dataset give clean code also save the plot as 'plot{cnt}.png' "
        }
    ],
    model="llama3-70b-8192",
    )
    viss = chat_completion.choices[0].message.content
    viss =  re.sub('python', '', viss)
    viss =  re.sub('Python', '', viss)
    pattern = r"```(.*?)```"

    # Find all matches
    matches = re.findall(pattern, viss, re.DOTALL)

    # Safely parse the response text as a Python AST (Abstract Syntax Tree)
    parsed_ast = ast.parse(matches[0])

    # else:
    # Execute the parsed AST
    exec(compile(parsed_ast, filename="<ast>", mode="exec"))
    
    return f"C:/Prodigy/DataGenie/plot{cnt}.png"



In [11]:
# def classify(mess) :
    
#     chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "system",
#             "content": f"""You are an assistant with knowledge on Data Science who classifies user prompt into one of the following categories:
#               visualization : if user need to visualize the data using any plots or charts,
#               searching : If the user is searcing for dataset available in online or he wants any dataset to be searched ,
#               data_cleaning : If the user wants to do any cleaning or preprocessing tasks ,
#               model_training : If the user wants to train a Model ,
#               suggestions : if the user wants to get some suggessions to what to do with the dataset  ,
#               Quering_dataset : If the user whats to check what is in the the dataset like unique values , datatypes , df.head() , or df.tail() . 
              
#               \n\n Don't give any kind of Explanation just give the classification """ ,
#         },
#         {
#             "role": "user",
#             "content": f"{mess}" ,
#         }
#     ],
#     model="llama3-70b-8192",
#     )
#     response_text = chat_completion.choices[0].message.content
#     return response_text

In [1]:

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/send_message', methods=['POST'])
def send_message():
    global cnt
    message = request.get_json()['message']
    cls = classify(message)
    print(cls)
    if "visualization" in cls :
        res = visualize(message)
        cnt += 1
        return jsonify({'response': "image" , "path" : res})
    elif "searching" in cls :
        res = search_dataset(message)

    elif 'data_cleaning' in cls or 'Quering_dataset' in cls :
        akg,msg = preprocess(message)
        res = akg+'\n\n' +msg
        # return res
    elif 'suggestions' in cls :
        res = suggest()
        # return res
    elif 'model_training' in cls :
        pass
        # res = suggest()
        # return res
    else :
        res = others(message)

    print(res)
    # if "visualize" in message.lower() or "plot" in message.lower() :
    #     res = visualize(message)
    #     cnt += 1
    #     return jsonify({'response': "image" , "path" : res})
    # response = process_message(message)
    return jsonify({'response': res })


def classify(mess) :
    
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": f"""You are an assistant with knowledge on Data Science who classifies user prompt into one of the following categories:
              visualization : if user need to visualize the data using any plots or charts,
              searching : If the user is searcing for dataset available in online or he wants any dataset to be searched ,
              data_cleaning : If the user wants to do any cleaning or preprocessing tasks ,
              model_training : If the user wants to train a Model ,
              suggestions : if the user wants to get some suggessions to what to do with the dataset  ,
              Quering_dataset : If the user whats to check what is in the the dataset like unique values , datatypes , df.head() , or df.tail() , 
              Others : If the query doesn't falls below any of the above classes like Hi , Hello and more .

              \n\n Don't give any kind of Explanation just give the classification """ ,
        },
        {
            "role": "user",
            "content": f"{mess}" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    return response_text
def process_message(message):
    # Implement your chatbot logic here
    res = ""
    if "search a dataset" in message.lower() :
        res = search_dataset(message)
        # return res

        # return res
    if 'preprocess' in message.lower() or 'clean' in message.lower() :
        res = preprocess(message)
        # return res
    if 'suggest' in message.lower() or 'next' in message.lower() :
        res = ()
        # return res
    else :
        res = others(message)
        
    return res
    return "image"


NameError: name 'app' is not defined

In [ ]:
@app.route('/download', methods=['GET'])
def download_csv():
    # Convert the DataFrame to a CSV string
    csv_string = df.to_csv(index=False)

    # Set the response headers
    response = make_response(csv_string)
    response.headers['Content-Disposition'] = 'attachment; filename="data.csv"'
    response.headers['Content-Type'] = 'text/csv'

    return response

In [14]:
if __name__ == '__main__':
    app.run(port=7000)


 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


127.0.0.1 - - [13/Jul/2024 13:13:42] "POST /process_file HTTP/1.1" 200 -


This dataset contains information about 5 customers, including their demographic characteristics such as gender, age, and annual income, as well as their spending habits represented by a spending score. The data aims to provide insights into the customers' behavior and preferences, potentially helping businesses to tailor their marketing strategies and improve customer engagement.


127.0.0.1 - - [13/Jul/2024 13:14:05] "OPTIONS /send_message HTTP/1.1" 200 -


Quering_dataset


127.0.0.1 - - [13/Jul/2024 13:14:06] "POST /send_message HTTP/1.1" 200 -


Here is the  code to get unique values in each column of the dataset and store the result in a variable named `result`:

```
result = {col: df[col].nunique() for col in df.columns}
print(result)

result_unique_values = {col: df[col].unique() for col in df.columns}
print(result_unique_values)
```
{'CustomerID': 200, 'Gender': 2, 'Age': 51, 'Annual Income (k$)': 64, 'Spending Score (1-100)': 84}
{'CustomerID': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 